# Ingest JSON from Raw to Bronze

Ingest bootstrap-static and fixtures from manually uploaded JSON files

Write to bronze layer tables in fpl_bronze volume.

In [0]:
from pyspark.sql import functions as F, types as T
from delta.tables import DeltaTable
from pyspark.sql import DataFrame, SparkSession

In [0]:
%%sql

select * from fpl_bronze_dev.fixtures_25_26

code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,pulse_id,started,stats,team_a,team_a_difficulty,team_a_score,team_h,team_h_difficulty,team_h_score,last_updated
2561895,1,true,true,1,2025-08-15T19:00:00Z,90,false,124791,true,"List(List(List(List(82, 2)), List(List(381, 1), List(384, 1), List(385, 1), List(661, 1)), goals_scored), List(List(List(86, 1), List(93, 1)), List(List(386, 1), List(392, 1), List(661, 1)), assists), List(List(), List(), own_goals), List(List(), List(), penalties_saved), List(List(), List(), penalties_missed), List(List(List(86, 1), List(97, 1)), List(List(371, 1)), yellow_cards), List(List(), List(), red_cards), List(List(List(67, 6)), List(List(366, 1)), saves), List(List(List(82, 3)), List(List(661, 2), List(381, 1)), bonus), List(List(List(82, 42), List(86, 23), List(84, 20), List(93, 12), List(85, 11), List(67, 10), List(90, 7), List(96, 3), List(73, 2), List(100, 2), List(685, 2), List(72, 1), List(74, -1), List(97, -1)), List(List(661, 37), List(381, 36), List(384, 34), List(385, 29), List(386, 27), List(387, 17), List(392, 15), List(382, 14), List(373, 11), List(366, 8), List(370, 8), List(374, 6), List(371, 5), List(388, 4), List(376, -1), List(372, -3)), bps), List(List(List(84, 20), List(85, 19), List(72, 15), List(73, 9), List(685, 9), List(86, 7), List(74, 6), List(82, 5), List(97, 5), List(93, 4), List(90, 3), List(77, 2)), List(List(373, 21), List(387, 11), List(381, 10), List(382, 9), List(384, 8), List(374, 7), List(371, 6), List(386, 6), List(370, 4), List(392, 3), List(372, 2), List(376, 1), List(385, 1), List(388, 1), List(661, 1)), defensive_contribution))",4,5,2,12,3,4,2025-10-26T14:14:13.363Z
2561896,1,true,true,2,2025-08-16T11:30:00Z,90,false,124792,true,"List(List(List(), List(), goals_scored), List(List(), List(), assists), List(List(), List(), own_goals), List(List(), List(), penalties_saved), List(List(), List(), penalties_missed), List(List(List(490, 1)), List(List(58, 1)), yellow_cards), List(List(), List(List(38, 1)), red_cards), List(List(List(469, 3)), List(List(33, 3)), saves), List(List(List(477, 3), List(469, 2), List(476, 1)), List(), bonus), List(List(List(477, 33), List(469, 32), List(476, 30), List(474, 26), List(478, 26), List(491, 20), List(487, 15), List(488, 11), List(486, 10), List(490, 7), List(485, 4), List(497, 4), List(489, 3), List(500, 3)), List(List(33, 29), List(36, 25), List(40, 25), List(37, 22), List(59, 19), List(38, 15), List(58, 13), List(64, 12), List(54, 10), List(47, 7), List(48, 5), List(53, 4)), bps), List(List(List(474, 11), List(476, 11), List(490, 9), List(491, 9), List(488, 8), List(477, 5), List(478, 4), List(485, 4), List(487, 4), List(486, 3), List(489, 2), List(497, 1)), List(List(59, 14), List(36, 11), List(58, 11), List(37, 7), List(47, 7), List(48, 7), List(54, 7), List(40, 5), List(64, 5), List(38, 3)), defensive_contribution))",15,4,0,2,3,0,2025-10-26T14:14:13.363Z
2561897,1,true,true,3,2025-08-16T14:00:00Z,90,false,124793,true,"List(List(List(List(338, 1)), List(List(164, 1)), goals_scored), List(List(List(329, 1)), List(List(158, 1)), assists), List(List(), List(), own_goals), List(List(), List(), penalties_saved), List(List(), List(), penalties_missed), List(List(List(318, 1), List(331, 1), List(338, 1)), List(List(148, 1), List(157, 1), List(169, 1)), yellow_cards), List(List(), List(), red_cards), List(List(List(314, 3)), List(List(139, 1)), saves), List(List(List(338, 3)), List(List(164, 2), List(166, 1)), bonus), List(List(List(338, 25), List(322, 17), List(329, 16), List(314, 15), List(317, 14), List(335, 11), List(321, 10), List(324, 10), List(332, 9), List(318, 3), List(319, 3), List(330, 3), List(325, 2), List(326, 1), List(331, -3)), List(List(164, 24), List(166, 20), List(173, 17), List(146, 16), List(151, 15), List(158, 15), List(167, 10), List(139, 9), List(145, 8), List(160, 7), List(169, 7), List(170, 3), List(178, 3), List(157, -2), List(162, -2), List(148, -5)), bps), Lis

DataFrame[code: bigint, event: bigint, finished: boolean, finished_provisional: boolean, id: bigint, kickoff_time: string, minutes: bigint, provisional_start_time: boolean, pulse_id: bigint, started: boolean, stats: array<struct<a:array<struct<element:bigint,value:bigint>>,h:array<struct<element:bigint,value:bigint>>,identifier:string>>, team_a: bigint, team_a_difficulty: bigint, team_a_score: bigint, team_h: bigint, team_h_difficulty: bigint, team_h_score: bigint, last_updated: timestamp]

In [0]:
def read_latest_raw_json(
        base_path: str, 
        filename: str, 
        spark: SparkSession, 
        utils) -> DataFrame:
    """
    Reads the latest raw JSON file from a folder structure in DBFS.

    Parameters:
    - base_path (str): Base path where folders are stored.
    - filename (str): Name of the JSON file to read.
    - spark (SparkSession): Active Spark session.
    - utils: utility object (databricks or fabric).

    Returns:
    - DataFrame: Parsed Spark DataFrame from the latest folder.
    """
    folders = utils.fs.ls(base_path)
    latest_folder = sorted(folders, key=lambda x: x.name, reverse=True)[0].path
    print(f"Loading folder: {latest_folder}")
    return spark.read.option("multiline", "true").json(f"{latest_folder}/{filename}")

def ingest_entity(
    entity_config: dict,
    bronze_schema: str,
    protocol: str,
    season: str,
    spark: SparkSession
) -> None:
    """
    Ingests a single entity into the bronze layer.

    Parameters:
    - entity_config (dict): Configuration for the entity.
    - bronze_schema (str): Target schema name.
    - protocol (str): Global ingestion protocol ('HIST' or 'INCR').
    - season (str): current season suffix for tables
    - spark (SparkSession): Active Spark session.

    The entity_config must include:
    - name (str): Entity name.
    - df (DataFrame): Source DataFrame.
    - protocol (str): Entity-specific protocol.
    Optional keys:
    - path (str): Column to select or explode.
    - explode (bool): Whether to explode the path.
    - alias (str): Alias for exploded column.
    - merge_key (str): Key to use for merge condition.
    """
    name = entity_config["name"] + "_" + season
    df = entity_config["df"]
    path = entity_config.get("path")
    explode = entity_config.get("explode", False)
    alias = entity_config.get("alias")
    merge_key = entity_config.get("merge_key")
    entity_protocol = entity_config["protocol"]

    # Extract and transform
    if explode and path:
        entity_df = df.select(F.explode(path).alias(alias)).select(f"{alias}.*")
    elif path:
        entity_df = df.select(path)
    else:
        entity_df = df

    # # Detect schema drift
    # detect_schema_drift(
    #     new_df=entity_df,
    #     table_name=f"{bronze_schema}.{name}",
    #     spark=spark
    # )
    
    # Write or merge
    if protocol == "HIST":
        write_to_table(
            df=entity_df,
            table_name=f"{bronze_schema}.{name}"
        )
        print(f"[HIST] {name} written to {bronze_schema}.{name}.")
    elif entity_protocol == "INCR" and protocol == "INCR":
        merge_to_table(
            df=entity_df,
            table_name=f"{bronze_schema}.{name}",
            merge_condition=f"target.{merge_key} = source.{merge_key}",
            spark=spark
        )
        print(f"[INCR] {name} merged to {bronze_schema}.{name}.")

from pyspark.sql import functions as F, types as T
from pyspark.sql import DataFrame

from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T

def normalize_fixtures_schema(df: DataFrame) -> DataFrame:
    """
    Normalizes the FPL fixtures 'stats' column so that every row conforms to
    Array<Struct<identifier: string, a: Array<Struct<element: bigint, value: bigint>>, h: Array<Struct<element: bigint, value: bigint>>>>.
    Works when some rows have [] or null and others have full structs.
    """

    # Define canonical schema for `stats`
    stats_struct_type = T.ArrayType(
        T.StructType([
            T.StructField("a", T.ArrayType(T.StructType([
                T.StructField("element", T.LongType(), True),
                T.StructField("value", T.LongType(), True)
            ])), True),
            T.StructField("h", T.ArrayType(T.StructType([
                T.StructField("element", T.LongType(), True),
                T.StructField("value", T.LongType(), True)
            ])), True),
            T.StructField("identifier", T.StringType(), True)
        ])
    )

    # Create an empty array of the correct type
    empty_array = F.lit([]).cast(stats_struct_type)

    # Normalize the stats column
    df = df.withColumn(
        "stats",
        F.when(
            (F.col("stats").isNull()) | (F.size(F.col("stats")) == 0),
            empty_array
        ).otherwise(F.col("stats").cast(stats_struct_type))
    )

    return df




In [0]:
def write_to_table(
    df: DataFrame,
    table_name: str,
    mode: str = "overwrite",
    merge_schema: bool = True,
    partition_by: list[str] = None,
    path: str = None,
    save_as_table: bool = True
) -> None:
    """
    Generalised Delta write helper for bronze layer.

    Parameters:
    - df (DataFrame): Spark DataFrame to write.
    - table_name (str): Name of the Delta table (used if save_as_table=True).
    - mode (str): Write mode ('overwrite', 'append', 'ignore', 'error', etc.).
    - merge_schema (bool): Whether to merge schema on write.
    - partition_by (list[str], optional): List of columns to partition by.
    - path (str, optional): Path to save the Delta table (used if save_as_table=False).
    - save_as_table (bool): If True, saves as managed table; else saves to path.

    Raises:
    - ValueError: If neither save_as_table nor path is properly specified.
    """

    df_with_ts = df.withColumn("last_updated", F.current_timestamp())

    writer = df_with_ts.write.format("delta").mode(mode)

    if merge_schema:
        writer = writer.option("mergeSchema", "true")
    elif mode == "overwrite":
        writer = writer.option("overwriteSchema", "true")

    if partition_by:
        writer = writer.partitionBy(*partition_by)

    if save_as_table:
        writer.saveAsTable(table_name)
    elif path:
        writer.save(path)
    else:
        raise ValueError("Either save_as_table must be True or a path must be provided.")

def detect_schema_drift(new_df: DataFrame, table_name: str, spark: SparkSession) -> bool:
    """
    Detects schema drift between a new DataFrame and an existing Delta table.

    Parameters:
    - new_df (DataFrame): The new DataFrame to compare.
    - table_name (str): The name of the existing Delta table.
    - spark (SparkSession): The active Spark session.

    Returns:
    - bool: True if schema drift is detected, False otherwise.
    """
    try:
        existing_df = spark.table(table_name)
        existing_fields = set(field.name for field in existing_df.schema.fields if field.name != "last_updated")
        new_fields = set(field.name for field in new_df.schema.fields if field.name != "last_updated")

        added = new_fields - existing_fields
        removed = existing_fields - new_fields

        if added or removed:
            print(f"Schema drift detected in {table_name}")
            if added:
                print(f"Added fields: {added}")
            if removed:
                print(f"Removed fields: {removed}")
            return True
        return False
    except Exception:
        print(f"ℹ No existing table found for {table_name}. Assuming first write.")
        return False

def merge_to_table(
    df: DataFrame,
    table_name: str,
    merge_condition: str,
    spark: SparkSession,
    partition_by: list[str] = None
) -> None:
    """
    Performs an upsert (merge) into a Delta table.

    Parameters:
    - df (DataFrame): Incoming DataFrame to merge.
    - table_name (str): Target Delta table name.
    - merge_condition (str): SQL condition for matching rows.
    - spark (SparkSession): Active Spark session.
    - partition_by (list[str], optional): Columns to partition by on initial write.

    If the table does not exist, it will be created using write_to_table.
    """
    df_with_ts = df.withColumn("last_updated", F.current_timestamp())

    if not spark.catalog.tableExists(table_name):
        write_to_table(
            df=df_with_ts,
            table_name=table_name,
            partition_by=partition_by
        )
    else:
        delta_table = DeltaTable.forName(spark, table_name)
        (
            delta_table.alias("target")
            .merge(
                source=df_with_ts.alias("source"),
                condition=merge_condition
            )
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )

In [0]:
try:
    ENV = dbutils.widgets.get("ENV")
except Exception:
    ENV = "dev"

try:
    PROTOCOL = dbutils.widgets.get("PROTOCOL")
except Exception:
    PROTOCOL = "INCR"

#ensure valid ENV and PROTOCOL
valid_envs = {"dev", "test", "prod"}
valid_protocols = {"HIST", "INCR"}

# Validate ENV
if ENV not in valid_envs:
    print(f"Invalid ENV: {ENV}. Must be one of {valid_envs}. Exiting notebook.")
    dbutils.notebook.exit("Invalid ENV")

# Validate PROTOCOL
if PROTOCOL not in valid_protocols:
    print(f"Invalid PROTOCOL: {PROTOCOL}. Must be one of {valid_protocols}. Exiting notebook.")
    dbutils.notebook.exit("Invalid PROTOCOL")

In [0]:
BRONZE_SCHEMA = f"fpl_bronze_{ENV}"
CURRENT_SEASON = "2025_26"
CURRENT_SEASON_SHORT = CURRENT_SEASON[2:]
BASE_RAW_JSON_PATH = f"/Volumes/workspace/fpl_raw/player_data/{CURRENT_SEASON}"

# Ingest Raw JSON files

bootstrap-static is core data, with schema as:

- events: Basic information of every Gameweek such as average score, highest score, top scoring player, most captained, etc. Incremental
- game_settings: The game settings and rules. 
- phases: Phases of FPL season. 
- teams: Basic information of current Premier League clubs.
- total_players: Total FPL players.
- elements: Information of all Premier League players including points, status, value, match stats (goals, assists, etc.), ICT index, etc. Incremental
- element_types: Basic information about player’s position (GK, DEF, MID, FWD).
- chips: All chips available in FPL.
- game_config: scoring and game setup rules.


fixtures contains all data about fixtures for the season. It needs to be incrementally loaded as fixtures change often due to clashes/TV viewing changes.

Ingested on a weekly basis (after end of gameweek) and tables written to {table_name}_{season} e.g. events_25_26 for future season ingestion.

In [0]:
bootstrap_static_df = read_latest_raw_json(
    base_path = BASE_RAW_JSON_PATH, 
    filename = "bootstrap_static.json",
    spark = spark,
    utils = dbutils
    )
    
fixtures_df = read_latest_raw_json(
    base_path = BASE_RAW_JSON_PATH, 
    filename = "fixtures.json",
    spark = spark,
    utils = dbutils
    )

fixtures_df = normalize_fixtures_schema(fixtures_df)

Loading folder: dbfs:/Volumes/workspace/fpl_raw/player_data/2025_26/gw_08/
Loading folder: dbfs:/Volumes/workspace/fpl_raw/player_data/2025_26/gw_08/


In [0]:
fixtures_df.printSchema()

root
 |-- code: long (nullable = true)
 |-- event: long (nullable = true)
 |-- finished: boolean (nullable = true)
 |-- finished_provisional: boolean (nullable = true)
 |-- id: long (nullable = true)
 |-- kickoff_time: string (nullable = true)
 |-- minutes: long (nullable = true)
 |-- provisional_start_time: boolean (nullable = true)
 |-- pulse_id: long (nullable = true)
 |-- started: boolean (nullable = true)
 |-- stats: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- a: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- element: long (nullable = true)
 |    |    |    |    |-- value: long (nullable = true)
 |    |    |-- h: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- element: long (nullable = true)
 |    |    |    |    |-- value: long (nullable = true)
 |    |    |-- identifier: string (nullable = true)
 |-- team_a: long (nullabl

In [0]:
display(fixtures_df)

code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,pulse_id,started,stats,team_a,team_a_difficulty,team_a_score,team_h,team_h_difficulty,team_h_score
2561895,1,true,true,1,2025-08-15T19:00:00Z,90,false,124791,true,"List(List(List(List(82, 2)), List(List(381, 1), List(384, 1), List(385, 1), List(661, 1)), goals_scored), List(List(List(86, 1), List(93, 1)), List(List(386, 1), List(392, 1), List(661, 1)), assists), List(List(), List(), own_goals), List(List(), List(), penalties_saved), List(List(), List(), penalties_missed), List(List(List(86, 1), List(97, 1)), List(List(371, 1)), yellow_cards), List(List(), List(), red_cards), List(List(List(67, 6)), List(List(366, 1)), saves), List(List(List(82, 3)), List(List(661, 2), List(381, 1)), bonus), List(List(List(82, 42), List(86, 23), List(84, 20), List(93, 12), List(85, 11), List(67, 10), List(90, 7), List(96, 3), List(73, 2), List(100, 2), List(685, 2), List(72, 1), List(74, -1), List(97, -1)), List(List(661, 37), List(381, 36), List(384, 34), List(385, 29), List(386, 27), List(387, 17), List(392, 15), List(382, 14), List(373, 11), List(366, 8), List(370, 8), List(374, 6), List(371, 5), List(388, 4), List(376, -1), List(372, -3)), bps), List(List(List(84, 20), List(85, 19), List(72, 15), List(73, 9), List(685, 9), List(86, 7), List(74, 6), List(82, 5), List(97, 5), List(93, 4), List(90, 3), List(77, 2)), List(List(373, 21), List(387, 11), List(381, 10), List(382, 9), List(384, 8), List(374, 7), List(371, 6), List(386, 6), List(370, 4), List(392, 3), List(372, 2), List(376, 1), List(385, 1), List(388, 1), List(661, 1)), defensive_contribution))",4,5,2,12,3,4
2561896,1,true,true,2,2025-08-16T11:30:00Z,90,false,124792,true,"List(List(List(), List(), goals_scored), List(List(), List(), assists), List(List(), List(), own_goals), List(List(), List(), penalties_saved), List(List(), List(), penalties_missed), List(List(List(490, 1)), List(List(58, 1)), yellow_cards), List(List(), List(List(38, 1)), red_cards), List(List(List(469, 3)), List(List(33, 3)), saves), List(List(List(477, 3), List(469, 2), List(476, 1)), List(), bonus), List(List(List(477, 33), List(469, 32), List(476, 30), List(474, 26), List(478, 26), List(491, 20), List(487, 15), List(488, 11), List(486, 10), List(490, 7), List(485, 4), List(497, 4), List(489, 3), List(500, 3)), List(List(33, 29), List(36, 25), List(40, 25), List(37, 22), List(59, 19), List(38, 15), List(58, 13), List(64, 12), List(54, 10), List(47, 7), List(48, 5), List(53, 4)), bps), List(List(List(474, 11), List(476, 11), List(490, 9), List(491, 9), List(488, 8), List(477, 5), List(478, 4), List(485, 4), List(487, 4), List(486, 3), List(489, 2), List(497, 1)), List(List(59, 14), List(36, 11), List(58, 11), List(37, 7), List(47, 7), List(48, 7), List(54, 7), List(40, 5), List(64, 5), List(38, 3)), defensive_contribution))",15,4,0,2,3,0
2561897,1,true,true,3,2025-08-16T14:00:00Z,90,false,124793,true,"List(List(List(List(338, 1)), List(List(164, 1)), goals_scored), List(List(List(329, 1)), List(List(158, 1)), assists), List(List(), List(), own_goals), List(List(), List(), penalties_saved), List(List(), List(), penalties_missed), List(List(List(318, 1), List(331, 1), List(338, 1)), List(List(148, 1), List(157, 1), List(169, 1)), yellow_cards), List(List(), List(), red_cards), List(List(List(314, 3)), List(List(139, 1)), saves), List(List(List(338, 3)), List(List(164, 2), List(166, 1)), bonus), List(List(List(338, 25), List(322, 17), List(329, 16), List(314, 15), List(317, 14), List(335, 11), List(321, 10), List(324, 10), List(332, 9), List(318, 3), List(319, 3), List(330, 3), List(325, 2), List(326, 1), List(331, -3)), List(List(164, 24), List(166, 20), List(173, 17), List(146, 16), List(151, 15), List(158, 15), List(167, 10), List(139, 9), List(145, 8), List(160, 7), List(169, 7), List(170, 3), List(178, 3), List(157, -2), List(162, -2), List(148, -5)), bps), List(List(List(335, 9), List(317, 8), List(322, 7), List(329, 6), 

In [0]:
ENTITY_CONFIG = [
    {
        "name": "chips",
        "df": bootstrap_static_df,
        "path": "chips",
        "explode": True,
        "alias": "chip",
        "protocol": "HIST"
    },
    {
        "name": "element_stats",
        "df": bootstrap_static_df,
        "path": "element_stats",
        "explode": True,
        "alias": "stat",
        "protocol": "HIST"
    },
    {
        "name": "element_types",
        "df": bootstrap_static_df,
        "path": "element_types",
        "explode": True,
        "alias": "type",
        "protocol": "HIST"
    },
    {
        "name": "game_config_scoring",
        "df": bootstrap_static_df,
        "path": "game_config.scoring",
        "season": CURRENT_SEASON_SHORT,
        "explode": False,
        "protocol": "HIST"
    },
    {
        "name": "game_config_rules",
        "df": bootstrap_static_df,
        "path": "game_config.rules",
        "explode": False,
        "protocol": "HIST"
    },
    {
        "name": "phases",
        "df": bootstrap_static_df,
        "path": "phases",
        "explode": True,
        "alias": "phase",
        "protocol": "HIST"
    },
    {
        "name": "teams",
        "df": bootstrap_static_df,
        "path": "teams",
        "explode": True,
        "alias": "team",
        "protocol": "HIST"
    },
    {
        "name": "elements",
        "df": bootstrap_static_df,
        "path": "elements",
        "explode": True,
        "alias": "player",
        "protocol": "INCR",
        "merge_key": "id"
    },
    {
        "name": "events",
        "df": bootstrap_static_df,
        "path": "events",
        "explode": True,
        "alias": "event",
        "protocol": "INCR",
        "merge_key": "id"
    },
    {
        "name": "fixtures",
        "df": fixtures_df,
        "path": None,
        "explode": False,
        "protocol": "INCR",
        "merge_key": "id"
    }

]

In [0]:
for entity in ENTITY_CONFIG:
    ingest_entity(
        entity_config = entity,
        bronze_schema = BRONZE_SCHEMA,
        protocol = PROTOCOL,
        season = CURRENT_SEASON_SHORT,
        spark = spark
    )

[INCR] elements_25_26 merged to fpl_bronze_dev.elements_25_26.
[INCR] events_25_26 merged to fpl_bronze_dev.events_25_26.
[INCR] fixtures_25_26 merged to fpl_bronze_dev.fixtures_25_26.
